In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

import folium
from folium import Choropleth
from folium import GeoJson
import geopandas as gpd

import plotly.express as px

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import KNNImputer

from sklearn.ensemble import RandomForestRegressor

In [2]:
pd.set_option('display.max_columns', None)

df = pd.read_csv('../bin/data_def.csv')

df.head()

,provincia,link_coche,fecha_extraccion,marca_modelo,fecha_matriculacion,kilometraje,nombre_vendedor,tipo_vendedor,distintivo_ambiental,garantia,precio_contado,precio_nuevo,largo,ancho,alto,capacidad_maletero,carroceria,num_plazas,batalla,peso,num_puertas,consumo_medio,consumo_carretera,consumo_urbano,CO2,deposito,combustible,cilindros,cilindrada,sobrealimentacion,tipo_cambio,traccion,num_marchas,potencia_kw,potencia_cv,par,velocidad_max,aceleracion
0,Madrid,https://www.autocasion.com/coches-segunda-mano...,25/11/2024,Detalles VOLKSWAGEN Polo (2019),Matriculado: 10/2019,93000,COCHES AMB,Profesional,C,12 meses,20950,29044,4053 mm Largo,1751 mm Ancho,1461 mm Alto,Capacidad del maletero (l)355/1.125,CarroceríaBerlina,Número de plazas5,Batalla (mm)\r\n2.548,Peso/Masa max. autorizado (kg)1.855,Número de puertas5,"Consumo medio 5,80 litros/100 km","Consumo en carretera 4,90 litros/100 km","Consumo urbano 7,70 litros/100 km",CO2 (g/km) 138 g/km,"Depósito de combustible (l) 40,00 litros",Combustible Gasolina,Nº de cilindros 4 cilindros,Cilindrada (cm3) 1.984,Sobrealimentación Turbo,Tipo de transmisión Automática secuencial,TracciónTracción delantera,Número de marchas6,Potencia (KW)147,Potencia (CV)200,Par (Nm)320,Velocidad máxima (km/h)237,"Aceleración 0-100 km/h (s)6,70"
1,Madrid,https://www.autocasion.com/coches-segunda-mano...,25/11/2024,Detalles OPEL Corsa (2009),Matriculado: 07/2009,236945,AUTOS CAPRICAR,Profesional,B,No,4000,15660,3999 mm Largo,1737 mm Ancho,1488 mm Alto,Capacidad del maletero (l)285/1.100,CarroceríaBerlina,Número de plazas5,Batalla (mm)\r\n2.511,Peso/Masa max. autorizado (kg)1.695,Número de puertas5,"Consumo medio 4,60 litros/100 km","Consumo en carretera 3,80 litros/100 km","Consumo urbano 5,90 litros/100 km",CO2 (g/km) 127 g/km,"Depósito de combustible (l) 45,00 litros",Combustible Diesel,Nº de cilindros 4 cilindros,Cilindrada (cm3) 1.248,Sobrealimentación Turbo,Tipo de transmisión Manual,TracciónTracción delantera,Número de marchas6,Potencia (KW)66,Potencia (CV)90,Par (Nm)200,Velocidad máxima (km/h)172,"Aceleración 0-100 km/h (s)12,70"
2,Murcia,https://www.autocasion.com/coches-segunda-mano...,25/11/2024,Detalles SKODA Octavia (2017),Matriculado: 08/2017,44300,SKOMOVIL MURCIA,Profesional,ECO,12 meses,15900,25030,4670 mm Largo,1814 mm Ancho,1458 mm Alto,Capacidad del maletero (l)460/1.450,CarroceríaBerlina,Número de plazas5,Batalla (mm)\r\n2.680,Peso/Masa max. autorizado (kg)1.878,Número de puertas5,"Consumo medio 5,70 litros/100 km","Consumo en carretera 4,60 litros/100 km","Consumo urbano 7,40 litros/100 km",CO2 (g/km) 101 g/km,"Depósito de combustible (l) 50,00 litros",Combustible Gasolina/gas,Nº de cilindros 4 cilindros,Cilindrada (cm3) 1.395,Sobrealimentación Turbo,Tipo de transmisión Manual,TracciónTracción delantera,Número de marchas6,Potencia (KW)81,Potencia (CV)110,Par (Nm)200,Velocidad máxima (km/h)195,"Aceleración 0-100 km/h (s)10,90"
3,Murcia,https://www.autocasion.com/coches-km0/km-0/sko...,25/11/2024,Detalles SKODA Kamiq (2024),Matriculado: 03/2024,10600,SKOMOVIL MURCIA,Profesional,C,12 meses,24900,no disponible,NaN,NaN,NaN,NaN,"4x4, SUV o pickup",NaN,NaN,NaN,5 Puertas,NaN,NaN,NaN,NaN,NaN,Gasolina,NaN,NaN,NaN,Automático,NaN,NaN,NaN,110,NaN,NaN,NaN
4,Murcia,https://www.autocasion.com/coches-segunda-mano...,25/11/2024,Detalles SKODA Octavia (2024),Matriculado: 02/2024,13673,SKOMOVIL MURCIA,Profesional,0 EMISIONES,12 meses,33500,no disponible,NaN,NaN,NaN,NaN,Familiar,NaN,NaN,NaN,5 Puertas,NaN,NaN,NaN,NaN,NaN,Eléctrico,NaN,NaN,NaN,Automático,NaN,NaN,NaN,204,NaN,NaN,NaN


## Limpieza preliminar

In [4]:
stats = []
for col in df.columns:
    stats.append((col, df[col].nunique(), df[col].isnull().sum() * 100 / df.shape[0], df[col].value_counts(normalize=True, dropna=False).values[0] * 100, df[col].dtype))
    stats_df = pd.DataFrame(stats, columns=['Columna', 'Valores únicos', '% nulos', '% of values in the biggest category', 'tipo'])
stats_df.sort_values('% nulos', ascending=False)

,Columna,Valores únicos,% nulos,% of values in the biggest category,tipo
24,CO2,216,19.030232,36.921279,object
37,aceleracion,138,17.555134,17.555134,object
36,velocidad_max,140,17.540529,17.540529,object
35,par,212,17.540529,17.540529,object
15,capacidad_maletero,875,17.540529,17.540529,object
18,batalla,325,17.540529,17.540529,object
33,potencia_kw,226,17.379874,17.379874,object
14,alto,413,17.365269,17.365269,object
13,ancho,287,17.365269,17.365269,object
12,largo,671,17.365269,17.365269,object


In [5]:
df['tipo_cambio'].unique()

#Hay nans que se toman como floats. Lo convierto a "no disponible"
df['tipo_cambio'] = df['tipo_cambio'].fillna("no disponible")

#No sé qué es manual automatizada, si queremos que sea manual hay que poner la lambda al revés
df['tipo_cambio'] = df['tipo_cambio'].apply(lambda x: 'Manual' if 'Manual' in x else 'Automático')

In [6]:
df["carroceria"] = df["carroceria"].str.replace("Carrocería","")

df["combustible"] = df["combustible"].str.replace("combustible ","").str.replace("Combustible ","")

In [7]:
df['fecha_matriculacion'] = df['fecha_matriculacion'].str.split("Matriculado: ").str[1]

def limpiar_fecha(fecha):
    try:
        mes, ano = fecha.split('/')
        return int(mes), int(ano)
    except:
        return np.nan, int(fecha)

df[['mes_matriculacion', 'ano_matriculacion']] = df['fecha_matriculacion'].apply(lambda x: pd.Series(limpiar_fecha(x)))

df = df.drop(columns=['fecha_matriculacion'])

In [8]:
df['mes_matriculacion'] = df['mes_matriculacion'].fillna(0).astype('int64')
df['ano_matriculacion'] = df['ano_matriculacion'].fillna(0).astype('int64')

In [9]:
df = df[df["tipo_vendedor"] == "Profesional"]
df = df.drop(columns=['tipo_vendedor'])

In [10]:
df["garantia"] = df["garantia"].str.split(' ').str[0].replace("Sí",np.nan).replace("No",0).astype(float)

In [11]:
df["precio_contado"] = pd.to_numeric(df["precio_contado"], errors='coerce')

df["precio_nuevo"] = pd.to_numeric(df["precio_nuevo"], errors='coerce')

df['largo'] = pd.to_numeric(df['largo'].str.extract('(\d+\.?\d*)')[0], errors='coerce')
df['ancho'] = pd.to_numeric(df['ancho'].str.extract('(\d+\.?\d*)')[0], errors='coerce')
df['alto'] = pd.to_numeric(df['alto'].str.extract('(\d+\.?\d*)')[0], errors='coerce')

df['capacidad_maletero'] = pd.to_numeric(df['capacidad_maletero'].str.extract(r'(\d+\.?\d*)')[0], errors='coerce')
df["num_plazas"] = df["num_plazas"].str.split("s").str[1].astype("Int64")
df["batalla"] = (df["batalla"].str.replace(r'Batalla \(mm\)\s*', '', regex=True).str.replace('.', '', regex=False))
df["batalla"] = pd.to_numeric(df["batalla"], errors='coerce').astype("Int64")
df["peso"] = df["peso"].str.replace('Peso/Masa max. autorizado (kg)', '').str.replace('.','').astype("Int64")
df["num_puertas"] = df["num_puertas"].str.extract(r'(\d+)').astype("Int64")
df["consumo_medio"] = df["consumo_medio"].str.extract(r'(\d+,\d+)').replace(',', '.', regex=True).astype(float)
df["consumo_carretera"] = df["consumo_carretera"].str.extract(r'(\d+,\d+)').replace(',', '.', regex=True).astype(float)
df["consumo_urbano"] = df["consumo_urbano"].str.extract(r'(\d+,\d+)').replace(',', '.', regex=True).astype(float)
df["co2"] = df["CO2"].str.extract(r'(\d{1,3})(?= g/km)')[0].astype(float)
df["num_cilindros"] = df["cilindros"].str.extract(r'(\d+)').astype("Int64")
df["cilindrada"] = df["cilindrada"].str.extract(r'(\d+,\d+|\d+\.\d+)', expand=False).str.replace(',', '').str.replace('.', '').astype("Int64")
df["deposito"] = df["deposito"].str.extract(r'(\d+,\d+)').replace(',', '.', regex=True).astype(float)

df["num_marchas"] = df["num_marchas"].str.extract(r'(\d+)').astype("Int64")
df["potencia_kw"] = df["potencia_kw"].str.extract(r'(\d+)').astype("Int64")
df["potencia_cv"] = df["potencia_cv"].str.extract(r'(\d+)').astype("Int64")
df["par"] = df["par"].str.extract(r'(\d+)').astype("Int64")
df["velocidad_max"] = df["velocidad_max"].str.extract(r'(\d+)').astype("Int64")
df["aceleracion"] = df["aceleracion"].str.extract(r'(\d+,\d+|\d+\.\d+)')[0].str.replace(",", ".").astype(float)
df['id_sobrealimentacion'] = df['sobrealimentacion'].apply(lambda x: x.replace('Sobrealimentación', '').replace('Tipo de sobrealimentador', 'no disponible').strip() if pd.notna(x) and x != '-' else 'no disponible')

In [12]:
df["traccion"].unique()

df["traccion"] = df['traccion'].fillna("no disponible")
df["traccion"] = df["traccion"].str.replace('Tracción', '').str.replace('Todo terreno tracción', '').apply(lambda x: x.strip())

In [13]:
marca_listado = pd.read_csv('../bin/listado_marcas.csv')

# Eliminamos EVO porque hay modelos de otras marcas (Land-Rover y Fiat) que contienen EVO en el nombre
# Lo añadimos al final para que coja primero el resto de marcas y añadimos un espacio al nombre por si acaso

lista_marcas = [marca for marca in marca_listado['nombre_marca'].tolist()]
#lista_marcas.remove("Evo")

lista_marcas_provisional = ["Cadillac", "Chevrolet", "Chrysler", "Dodge", "Infiniti", "EVO "]
for marca in lista_marcas_provisional:
    lista_marcas.append(marca)
lista_marcas = [marca.upper() for marca in lista_marcas]

df['marca_modelo'] = df['marca_modelo'].str.replace("Detalles ", "").str.strip().str.upper()

df['marca'] = df['marca_modelo'].apply(lambda x: next((marca for marca in lista_marcas if marca in x), np.nan))
df['modelo'] = df.apply(lambda x: re.sub(r'\s*\(.*?\)', '', x['marca_modelo'].replace(x['marca'], "").strip()).strip() if pd.notnull(x['marca']) else x['marca_modelo'], axis=1)

In [14]:
listado_marcas = {}

for index,detalle in enumerate (lista_marcas, start = 1):
    detalle = detalle.strip()
    
    listado_marcas[index] = detalle
    
df_marcas = pd.DataFrame(list(listado_marcas.items()), columns=['id_marca', 'nombre_marca'])

df_marcas.to_csv('../bin/listado_marcas.csv', index=False)

In [15]:
df = df.drop(columns=['marca_modelo', 'CO2', 'cilindros', 'sobrealimentacion', 'carroceria'])

In [16]:
df.rename(columns={'distintivo_ambiental': 'id_distintivo_ambiental'
                  , 'marca': 'id_marca'
                  , 'modelo': 'id_modelo'
                  , 'nombre_vendedor': 'id_concesionario'
                  , 'provincia': 'id_provincia'
                  , 'traccion': 'id_traccion'}, inplace=True)

In [17]:
df['combustible'] = (
    df['combustible']
    .str.replace('-', 'no disponible')
    .str.replace('Diésel', 'Diesel')
    .str.replace('Gasolina y corriente eléctrica', 'Híbrido Enchufable', regex=False)
    .str.replace('Diesel y corriente eléctrica', 'Híbrido Enchufable', regex=False)
    .str.replace('Corriente eléctrica', 'Eléctrico', regex=False)
    .str.replace(r'\bHíbrido\b(?!.*Enchufable)', 'Híbrido Enchufable', regex=True)
)

In [18]:
df['id_concesionario'] = df['id_concesionario'].apply(lambda x: x.split('-')[0])
df['id_concesionario'] = df['id_concesionario'].apply(lambda x: x.split(',')[0])

In [19]:
for columna in df.columns:
    df[columna] = df[columna].replace(["no disponible", "<NA>", "-", "0", 0, np.nan], None)
    

In [20]:
df = df[df['precio_contado'].notna()]

## Rtdo limpieza

In [22]:
df.sample(10)

,id_provincia,link_coche,fecha_extraccion,kilometraje,id_concesionario,id_distintivo_ambiental,garantia,precio_contado,precio_nuevo,largo,ancho,alto,capacidad_maletero,num_plazas,batalla,peso,num_puertas,consumo_medio,consumo_carretera,consumo_urbano,deposito,combustible,cilindrada,tipo_cambio,id_traccion,num_marchas,potencia_kw,potencia_cv,par,velocidad_max,aceleracion,mes_matriculacion,ano_matriculacion,co2,num_cilindros,id_sobrealimentacion,id_marca,id_modelo
1102,Madrid,https://www.autocasion.com/coches-segunda-mano...,25/11/2024,220768,House Cars Luxury,C,12.0,7000.0,22850.0,4060.0,1765.0,1435.0,309.0,5,2538,1620,5,None,None,None,44.0,Gasolina,1199,Manual,delantera,6,74,100,205,188,9.9,12,2019,None,3,None,OPEL,CORSA
6743,Madrid,https://www.autocasion.com/coches-km0/km-0/cup...,26/11/2024,50,GRUPO URETA AUTOMÓVILES,C,36.0,33900.0,39160.0,4446.0,1839.0,1520.0,450.0,5,2679,2030,5,None,None,None,50.0,Diesel,1968,Manual,delantera,6,110,150,340,205,9.3,6,2024,None,4,Turbo,CUPRA,FORMENTOR
1308,Murcia,https://www.autocasion.com/coches-segunda-mano...,25/11/2024,15803,TOYOTA LABASA MURCIA,C,12.0,18900.0,None,None,None,None,None,None,None,None,5,None,None,None,None,Gasolina,None,Manual,None,None,None,131,None,None,None,7,2020,None,None,None,CITROEN,C5 AIRCROSS
5526,Madrid,https://www.autocasion.com/coches-segunda-mano...,26/11/2024,36170,BYmyCAR Madrid,C,12.0,23901.0,27250.0,3821.0,1727.0,1415.0,160.0,4,2495,1670,2,5.4,4.6,6.8,40.0,Gasolina,1499,Manual,delantera,6,100,136,220,208,8.8,6,2019,126.0,3,None,MINI,
4610,Madrid,https://www.autocasion.com/coches-segunda-mano...,26/11/2024,26356,AUTO ELIA,0 EMISIONES,24.0,40900.0,53743.0,4440.0,1863.0,1652.0,452.0,5,2702,2290,5,None,None,None,48.0,Híbrido Enchufable,1477,Automático,delantera,7,155,211,245,180,8.5,4,2023,None,3,Turbo,VOLVO,XC40
5185,Madrid,https://www.autocasion.com/coches-segunda-mano...,26/11/2024,54699,AUTOMOVILES CANALCAR,C,12.0,12450.0,21090.0,4135.0,1765.0,1565.0,354.0,5,2530,1770,5,4.0,3.7,4.6,46.0,Diesel,1461,Manual,delantera,6,81,110,260,175,11.2,5,2016,104.0,4,Turbo,NISSAN,JUKE
6786,Jaén,https://www.autocasion.com/coches-km0/km-0/hyu...,26/11/2024,10,JADISA,ECO,60.0,25990.0,28650.0,4340.0,1795.0,1455.0,395.0,5,2650,1800,5,None,None,None,50.0,Gasolina,None,Manual,delantera,6,74,100,172,178,13.1,9,2024,None,3,Turbo,HYUNDAI,I30
5603,Madrid,https://www.autocasion.com/coches-segunda-mano...,26/11/2024,55897,AUTOMOVILES CANALCAR,C,12.0,26490.0,48339.0,4371.0,2090.0,1660.0,575.0,5,2660,2375,5,5.1,3.6,6.6,63.0,Diesel,1999,Automático,total permanente,9,110,150,430,180,10.0,9,2019,164.0,4,Turbo de geometría variable,LAND-ROVER,RANGE ROVER EVOQUE
6826,Jaén,https://www.autocasion.com/coches-km0/km-0/hyu...,26/11/2024,10,JADISA,ECO,60.0,25990.0,28650.0,4340.0,1795.0,1455.0,395.0,5,2650,1800,5,None,None,None,50.0,Gasolina,None,Manual,delantera,6,74,100,172,178,13.1,9,2024,None,3,Turbo,HYUNDAI,I30
3819,Málaga,https://www.autocasion.com/coches-segunda-mano...,26/11/2024,13964,Holamotor Málaga,C,12.0,25900.0,31056.0,4029.0,1740.0,1433.0,335.0,5,2563,1655,5,4.8,4.1,5.9,40.0,Gasolina,None,Manual,delantera,6,81,110,200,203,9.5,10,2023,110.0,3,Turbo,AUDI,A1


In [23]:
stats = []
for col in df.columns:
    stats.append((col, df[col].nunique(), df[col].isnull().sum() * 100 / df.shape[0], df[col].value_counts(normalize=True, dropna=False).values[0] * 100, df[col].dtype))
    stats_df = pd.DataFrame(stats, columns=['Columna', 'Valores únicos', '% nulos', '% of values in the biggest category', 'tipo'])
stats_df.sort_values('% nulos', ascending=False)

,Columna,Valores únicos,% nulos,% of values in the biggest category,tipo
33,co2,207,56.208236,56.208236,object
19,consumo_urbano,142,56.161616,56.161616,object
18,consumo_carretera,82,56.161616,56.161616,object
17,consumo_medio,114,50.458430,50.458430,object
35,id_sobrealimentacion,5,34.079254,43.045843,object
22,cilindrada,199,28.189588,28.189588,object
20,deposito,81,24.413364,24.413364,object
34,num_cilindros,7,23.512044,46.511267,object
12,capacidad_maletero,359,19.906760,19.906760,object
29,velocidad_max,137,19.704740,19.704740,object


In [24]:
df.to_csv('../bin/data_preprocess.csv', index=False)

In [25]:
df[df['id_marca'] == 'EVO']

,id_provincia,link_coche,fecha_extraccion,kilometraje,id_concesionario,id_distintivo_ambiental,garantia,precio_contado,precio_nuevo,largo,ancho,alto,capacidad_maletero,num_plazas,batalla,peso,num_puertas,consumo_medio,consumo_carretera,consumo_urbano,deposito,combustible,cilindrada,tipo_cambio,id_traccion,num_marchas,potencia_kw,potencia_cv,par,velocidad_max,aceleracion,mes_matriculacion,ano_matriculacion,co2,num_cilindros,id_sobrealimentacion,id_marca,id_modelo
6571,Vizcaya,https://www.autocasion.com/coches-km0/km-0/evo...,26/11/2024,4594,MEURI OCASION,C,25.0,19990.0,20900.0,4325.0,1765.0,1640.0,600.0,5,2560,1650,5,None,None,None,42.0,Gasolina,1590,Manual,delantera,6,84,114,155,180,10.5,1,2024,None,4,None,EVO,4
6584,Vizcaya,https://www.autocasion.com/coches-km0/km-0/evo...,26/11/2024,None,MEURI OCASION,C,36.0,21900.0,16101.0,4420.0,1760.0,1670.0,550.0,5,2510,1980,5,None,None,None,55.0,Gasolina,1598,Manual,delantera,5,93,127,149,175,11.5,2,2024,None,4,None,EVO,5
6585,Vizcaya,https://www.autocasion.com/coches-km0/km-0/evo...,26/11/2024,None,MEURI OCASION,ECO,36.0,21900.0,None,None,None,None,None,None,None,None,5,None,None,None,None,Gas,1600,Manual,None,None,None,106,None,None,None,2,2024,None,None,None,EVO,4
6590,Vizcaya,https://www.autocasion.com/coches-km0/km-0/evo...,26/11/2024,None,MEURI OCASION,C,36.0,20900.0,20900.0,4325.0,1765.0,1640.0,600.0,5,2560,1650,5,None,None,None,42.0,Gasolina,1590,Manual,delantera,6,84,114,155,180,10.5,2,2024,None,4,None,EVO,4
